In [1]:
import os
os.getcwd()

'/Users/anwesha/Documents/STUDY DOCUMENTS/ARUNANGSHU/Kaggle dataset/Ames Housing Dataset'

In [3]:
os.chdir('/Users/anwesha/Documents/STUDY DOCUMENTS/ARUNANGSHU/Kaggle dataset/Ames Housing Dataset')

In [4]:
os.getcwd()

'/Users/anwesha/Documents/STUDY DOCUMENTS/ARUNANGSHU/Kaggle dataset/Ames Housing Dataset'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
import sklearn.preprocessing as preprocessing
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train.shape

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False, cmap='viridis')

In [ ]:
train.dropna(subset=['SalePrice'],axis=0,inplace=True)

In [ ]:
X=train.copy()
y=X.pop('SalePrice')
model=RandomForestRegressor(n_estimators=100,random_state=0)

def MAE_calculation(X,y,model):
    "Step-1 Replacement of NaN"
    from sklearn.compose import ColumnTransformer
    column_trans = ColumnTransformer(
        [(X.select_dtypes(['float64','int64']),SimpleImputer(strategy='most_frequent')),
         (X.select_dtypes(['object']), SimpleImputer(strategy='most_frequent'))],
        remainder='passthrough')
    
    "Step-2*** Factorization of categorical columns"
    for colname in X.select_dtypes(["category", "object"]):
        X[colname],_ = X[colname].factorize()
        
    "Step-3*** Train test & split"
    train_X, val_X, train_y, val_y = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)
    
    "Step-4*** Final Step of error calculation "
    model.fit(train_X,train_y)
    
    #scores =cross_val_score(model.predict(X),y, cv=3, scoring='mean_absolute_error')
    
    print("MAE is:",mean_absolute_error(model.predict(val_X),val_y))

In [ ]:
X.drop(columns=['LotFrontage','MasVnrArea','GarageYrBlt'],inplace=True)
MAE_calculation(X,y,model)

In [ ]:
MAE_calculation(X,y,model=KNeighborsRegressor())

In [ ]:
discrete_features = X.dtypes == int

from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::10] 

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 15))
plot_mi_scores(mi_scores)

In [ ]:
def compare_object_columns(train,test):
    l=train.select_dtypes(include='object').columns.tolist()
    for i in l:
        Tr=train[i].unique().tolist()
        Te=test[i].unique().tolist()
        if set(train[i].unique())!=set(test[i].unique()):
            Tr=["nan" if x is np.nan else x for x in Tr]
            print ('**** train:',i,sorted(set(Tr)),end='\n')
            Te=["nan" if x is np.nan else x for x in Te]
            print ('#### test:',i,sorted(set(Te)),end='\n''\n')

In [ ]:
compare_object_columns(train,test)

In [ ]:
train.columns

In [ ]:
len(train)

In [ ]:
for cols in train.select_dtypes(include='object'):
    if set(train[cols].unique())!=set(test[cols].unique()):
        print ('**',cols,'**','\n','\n',train.groupby(cols)[cols].agg('count'),'**/n**')
    

In [ ]:
"from above two studies we see that MiscFeature: TenC=1, PoolQC: Fa=2, GarageQual: Ex=3, Eelctrical: Mix=1, Heating: Floor &" 
"OthW are 1 & 2, Exterior2nd: Other=1, Exterior1st: Imstucc & Stone 1 & 2, RoomMatl: ClyTile=1, Membran=1, Metal=1 & Roll=1"
"HouseStyle 2.5Fin=8, Condition2:RRAe=1, RRAn=1, RRNn=2, Utilities: NoSeWa=1 // In short these are in very small qty. Hence"
"Hence we are dropping all these rows from train dataset. Point to be noted that these are not there in test dataset also"

In [ ]:
train1=train[(train.MiscFeature !='TenC') & (train.PoolQC !='Fa') & (train.GarageQual !='Ex') 
              & (train.Electrical !='Mix') & (train.Heating.isin(['Floor','OthW'])==False) & 
             (train.Exterior1st.isin(['ImStucc','Stone'])==False)
             & (train.Exterior2nd !='Other') & (train.RoofMatl.isin(['ClyTile','Membran','Metal','Roll'])==False)
             & (train.HouseStyle !='2.5Fin') & (train.Condition2.isin(['RRAe','RRAn','RRNn'])==False) & (train.Utilities !='NoSeWa')]

In [ ]:
train1.shape

In [ ]:
train1.BedroomAbvGr.unique()

In [ ]:
"Exploring the 0 bedroom sizes: which clearly does not make any sense"

train[train.BedroomAbvGr==0].shape

"So we have 6 such cases, which we are dropping"

train2=train1[(train1.BedroomAbvGr!=0)]


In [ ]:
train2.shape

In [ ]:
"Let us explore the construction area , sales price and bedroom size"

In [ ]:
train2.loc[:,('LotArea','BsmtUnfSF','TotalBsmtSF','1stFlrSF', '2ndFlrSF','GarageArea','GrLivArea','WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch','3SsnPorch', 'ScreenPorch', 'PoolArea')]

In [ ]:
train2['Total_SF']=train2[['TotalBsmtSF','1stFlrSF', '2ndFlrSF','GarageArea','GrLivArea','WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch','3SsnPorch', 'ScreenPorch', 'PoolArea']].sum(axis=1)

test['Total_SF']=test[['TotalBsmtSF','1stFlrSF', '2ndFlrSF','GarageArea','GrLivArea','WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch','3SsnPorch', 'ScreenPorch', 'PoolArea']].sum(axis=1)

test2=test.copy()

In [ ]:
train2['Per_SF_Rate']=(train2['SalePrice']/train2['Total_SF']).round(0)

In [ ]:
train2.head(2)

In [ ]:
train.loc[:,('PoolQC','PoolArea')]

In [ ]:
test.groupby('PoolQC')['PoolQC'].agg('count')

In [ ]:
test.groupby('PoolArea')['PoolArea'].agg('count')

In [ ]:
"Pool QC data points are very low; we also see that these are very low in test data set also. Hence we are dropping PoolQC & PoolArea"

In [ ]:
train3=train2.drop(['PoolQC','PoolArea'],axis=1)
test3=test2.drop(['PoolQC','PoolArea'],axis=1)

In [ ]:
train3.groupby('BedroomAbvGr')['BedroomAbvGr'].agg('count')

In [ ]:
train3.BedroomAbvGr.unique()

In [ ]:
train3[train3.BedroomAbvGr==1].Total_SF.describe()

In [ ]:
train3[train3.BedroomAbvGr==2].Total_SF.describe()


In [ ]:
train3[train3.BedroomAbvGr==3].Total_SF.describe()

In [ ]:
train3[train3.BedroomAbvGr==4].Total_SF.describe()

In [ ]:
train3[train3.BedroomAbvGr==5].Total_SF.describe()

In [ ]:
train3[train3.BedroomAbvGr==6].Total_SF.describe()

In [ ]:
train3.shape

In [ ]:
def outlier_removal_Neighborhood_SF_Rate(df):
    df1=pd.DataFrame()
    for key, df2 in df.groupby('Neighborhood'):
        mean=np.mean(df2.Per_SF_Rate)
        std=np.std(df2.Per_SF_Rate)
        df3=df2[(df2.Per_SF_Rate>(mean-2*std)) & (df2.Per_SF_Rate<(mean+2*std))]
        df1=pd.concat([df1,df3],ignore_index=True)
    return df1

In [ ]:
train4=outlier_removal_Neighborhood_SF_Rate(train3)

In [ ]:
train4.shape

In [ ]:
def outlier_removal_Bedroom_Total_SF(df):
    df1=pd.DataFrame()
    for key, df2 in df.groupby('BedroomAbvGr'):
        mean=np.mean(df2.Total_SF)
        std=np.std(df2.Total_SF)
        df3=df2[(df2.Total_SF>(mean-2*std)) & (df2.Total_SF<(mean+2*std))]
        df1=pd.concat([df1,df3],ignore_index=True)
    return df1

In [ ]:
train5=outlier_removal_Bedroom_Total_SF(train4)

In [ ]:
train5.shape

In [ ]:
df=train5.copy()
l=df.select_dtypes(include=['object','int64','float64']).columns

results=[]

for i in l:
    sum_na=df[i].isna().sum()
    sum_na += np.sum(df[i] == "nan")
    if sum_na>0:
        print(i,',',str(sum_na),',',round((sum_na/len(df))*100),'%')
        results.append(i)
sns.barplot(y=df[results].isnull().sum().index, x=df[results].isnull().sum().values)
plt.show()

In [ ]:
def plot_scatter_chart(df,Neighborhood):
    bed2=df[(df.Neighborhood==Neighborhood) & (df.BedroomAbvGr==2)]
    bed3=df[(df.Neighborhood==Neighborhood) & (df.BedroomAbvGr==3)]
    plt.figure(dpi=100, figsize=(5,5))
    plt.scatter(bed2.Total_SF,bed2.SalePrice/1000,color='blue',label='2 Bedroom', s=10)
    plt.scatter(bed3.Total_SF,bed3.SalePrice/1000,color='red',label='3 Bedroom', s=10)
    plt.xlabel('Total Square Feet Area')
    plt.ylabel('SalePrice in kilo USD')
    plt.title(Neighborhood)
    plt.legend()

plot_scatter_chart(train5,'NAmes')

In [ ]:
def remove_Price_Per_SF_outliers(df):
    remove_rows=np.array([])
    for locality, location_df in df.groupby('Neighborhood'):
        bedroom_stat={}
        for bedroom, bedroom_df in df.groupby('BedroomAbvGr'):
            bedroom_stat[bedroom]={
                'mean': np.mean(bedroom_df.Per_SF_Rate),
                'std': np.std(bedroom_df.Per_SF_Rate),
                'count': bedroom_df.shape[0]
            }
        for bedroom, bedroom_df in df.groupby('BedroomAbvGr'):
            stat=bedroom_stat.get(bedroom-1)
            if stat and stat['count']>5:
                remove_rows=np.append(remove_rows, bedroom_df[bedroom_df.Per_SF_Rate<(stat['mean'])].index.values)
    return df.drop(remove_rows,axis='index')

In [ ]:
train6=remove_Price_Per_SF_outliers(train5)

In [ ]:
train6.shape

In [ ]:
plot_scatter_chart(train6,'NAmes')

In [ ]:
plt.hist(train6.Per_SF_Rate,rwidth=0.5)
plt.xlabel('Price Per SF')
plt.ylabel('Count')

In [ ]:
train7=train6.drop(['Per_SF_Rate','MiscFeature','Alley','FireplaceQu','Fence','LotFrontage'],axis=1)
train7.head(2)
test4=test3.drop(['MiscFeature','Alley','FireplaceQu','Fence','LotFrontage'],axis=1)

In [ ]:
train7.shape

In [ ]:
#train8=train7.dropna(subset=['GarageType','GarageYrBlt', 'GarageFinish','GarageQual','GarageCond',
#                             'MasVnrType','MasVnrArea','BsmtQual','BsmtCond','BsmtExposure',
#                             'BsmtFinType1','BsmtFinType2','Electrical'],axis=0)
#train8.shape

"We tried by removing the NaNs from rows instaed of dropping these columns, the result from grid search" 
"cv is as mentioned below"



In [ ]:
"Insetad of removing the NaNs if we drop the column the result has increased significantly"
"Hence we follow the same approach in test dataset also"

train8=train7.drop(['GarageType','GarageYrBlt', 'GarageFinish','GarageQual','GarageCond',
                             'MasVnrType','MasVnrArea','BsmtQual','BsmtCond','BsmtExposure',
                             'BsmtFinType1','BsmtFinType2','Electrical'],axis=1)
train8.shape

test5=test4.drop(['GarageType','GarageYrBlt', 'GarageFinish','GarageQual','GarageCond',
                             'MasVnrType','MasVnrArea','BsmtQual','BsmtCond','BsmtExposure',
                             'BsmtFinType1','BsmtFinType2','Electrical'],axis=1)
test5.shape

In [ ]:
train8.shape

In [ ]:
num_cols=test5.select_dtypes(exclude='object')
cat_cols=test5.select_dtypes(include='object')

num_impute=SimpleImputer(strategy='mean')
cat_impute=SimpleImputer(strategy='most_frequent')

num_rev=num_impute.fit_transform(num_cols)
cat_rev=cat_impute.fit_transform(cat_cols)

num_df=pd.DataFrame(num_rev,columns=num_cols.columns)
cat_df=pd.DataFrame(cat_rev,columns=cat_cols.columns)

test6=pd.concat([num_df,cat_df],axis=1)

test6.shape

In [ ]:
"Model Preparation"

dummy_train=pd.get_dummies(train8.select_dtypes(include='object'))
dummy_train.shape

dummy_test=pd.get_dummies(test5.select_dtypes(include='object'))
dummy_test.shape


In [ ]:
dummy_train.shape

In [ ]:
categorical_cols=[cols for cols in test5.columns if test5[cols].dtype=='object'  
                 and set(test5[cols]).issubset(set(train8[cols]))] 
len(categorical_cols)

In [ ]:
#test6.select_dtypes(include='object')

In [ ]:
'Neighborhood','Exterior1st','Exterior2nd'
'while we can drop exterior 1st & 2nd we cannot drop neighborhood'

In [ ]:
compare_object_columns(train8,test6)

In [ ]:
train9=pd.concat([train8,dummy_train],axis=1)
train9.shape

test6=pd.concat([test5,dummy_test],axis=1)
test6.shape

In [ ]:
train9.shape

In [ ]:
columns_to_drop_train=train9.select_dtypes(include='object')
train10=train9.drop(columns=columns_to_drop_train,axis=1)
train10.shape

columns_to_drop_test=test6.select_dtypes(include='object')
test7=test6.drop(columns=columns_to_drop_test,axis=1)
test7.shape

In [ ]:
test7.head(2)

In [ ]:
train10.shape

In [ ]:
X=train10.copy()
y=X.pop('SalePrice')

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
lm.fit(train_X,train_y)
lm.score(val_X,val_y)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv=ShuffleSplit(n_splits=5,test_size=0.2, random_state=0)

cross_val_score(LinearRegression(),X,y,cv=cv)

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

def find_best_model_Gridsearchcv (X,y):
    
    algo={
        'Linear Regression': {
            'model': LinearRegression(),
            'params': {
                'normalize': [True,False]
            }
        },
        'Lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random','cyclic']
            }
        },
        
        'Decision Tree':{
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
       'Random Forest':{
            'model': RandomForestRegressor(),
            'params': {
                'criterion': ['mse'],
                'n_estimators': [100]
                
            }
       },
        
        'K Nearest Neighbors':{
            'model': KNeighborsRegressor(),
            'params':{
                'n_neighbors':[5]
            }
        },
        
        'XGBRegressor':{
            'model': XGBRegressor(),
            'params':{
                'n_estimators':[50],
                
            }
        },
    }
    
    scores=[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name, config in algo.items():
        gs=GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model':algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_Gridsearchcv (X,y)

In [ ]:
test7.shape

In [ ]:
train10.shape

In [ ]:
final_train, final_test = X.align(test7, join='left', axis=1, fill_value=0)  # inner join

In [ ]:
from sklearn.linear_model import Lasso
lasso=Lasso()
lasso.fit(train_X,train_y)
test_pred=lasso.predict(final_test)

In [ ]:
output = pd.DataFrame({'Id': test7.Id,
                       'SalePrice': test_pred})
output.to_csv('Ames_Housing_Submissio_Rev-1,Date-01.12.2022.csv', index=False)

In [ ]:
from xgboost import XGBRegressor
xgb=XGBRegressor()
xgb.fit(train_X,train_y)
test_pred_xgb=xgb.predict(final_test)

In [ ]:
output = pd.DataFrame({'Id': test7.Id,
                       'SalePrice': test_pred_xgb})
output.to_csv('Ames_Housing_Submissio_Rev-2,Date-01.12.2022.csv', index=False)